In [1]:
import numpy as np

#import scipy.signal
#from scipy.integrate import odeint

import matplotlib as mp
#import matplotlib.ticker
from matplotlib.widgets import Cursor
#default_colors = mp.rcParams['axes.prop_cycle']

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams.update({
    'font.family' : 'sans-serif',
    #'mathtext.default' : 'rm',
    'mathtext.fontset': 'cm'
})
#mp.rcParams['text.usetex'] = True

import pandas as pd

#from lmfit import minimize, Parameters, Parameter, report_fit, fit_report
#from lmfit.model import Model, save_model, load_model, ModelResult

import os

#import pickle

#from mymodules.Excitation import excitation
#from mymodules.data_filtering import filt

%matplotlib qt5
%matplotlib

Using matplotlib backend: Qt5Agg


# Plot of the PL data in a file

## Plot of the PL as a function of wavelength

In [2]:
# Importing the data using pandas
path_pl = r"example_PL_plot.txt" #  r for those using Windows and \ in the path
data = pd.read_csv(path_pl, sep=",", header=None, index_col=False)
data.columns=["Wavelength", "Intensity"]
wavelen = data["Wavelength"]
pl = data["Intensity"]

# The signal will be averaged between lambda1 and lambda2 to calculate the background
lambda1 = 400
lambda2 = 450
lim1 = 1
while wavelen[lim1] < lambda1:
    lim1 += 1
lim2 = lim1
while wavelen[lim2] < lambda2:
    lim2 += 1
pl = pl - pl[lim1:lim2].sum()/(lim2-lim1)
pl_norm = pl/pl.max()

plt.close()
fig, ax = plt.subplots()
ax.plot(wavelen, pl_norm)

# Ticks" parameters
xmin = 430
xmax = 620 #for xlim
xstep_maj = 50 #for xticks
xstep_min = 10
majorLocator = mp.ticker.MultipleLocator(xstep_maj)
minorLocator = mp.ticker.MultipleLocator(xstep_min)
#ymin = 0
#ymax = 1.15

# Axes" limits
ax.set_xlim(xmin, xmax)
#ax.set_ylim(ymin, ymax)

# Titles
ax.set_title(r"PL($\lambda$)", fontsize=20, verticalalignment="bottom")
ax.set_xlabel("wavelength (nm)", fontsize=15)
ax.set_ylabel("PL (normalized)", fontsize=15, verticalalignment="bottom")

# Ticks
xticks = np.arange(xmin, xmax, xstep_min)
ax.set_xticks(xticks, minor=True)
ax.tick_params(right=False, which="both", labelsize=14)
ax.tick_params(which="major", width=.90, length=5)
ax.tick_params(which="minor", width=.75, length=3.5)
ax.xaxis.set_major_locator(majorLocator)
ax.xaxis.set_minor_locator(minorLocator)

# Legend
labels=["data"]
ax.legend(labels, fontsize=18)

fig.tight_layout(rect=(0, 0, 0.95, 0.95))

plt.show()

## Plot of the PL as a function of energy

In [3]:
# Importing the data using numpy
path_pl = r"example_PL_plot.txt" #  r for those using Windows and \ in the path
wavelen, pl = np.loadtxt(path_pl, delimiter=",").T

h = 6.626e-34
c = 3e8
e = 1.6e-19
E = (h*c/(wavelen*1e-9))/e
#E = E[::-1] #  for the elements of E to be in the ascending order
#pl = pl[::-1]

# The signal will be averaged between E1 and E2 to calculate the background
E1 = 1.8
E2 = 2
lim2 = 1
while E[lim2] > E2: #  > because the elements of E are in the descending order
    lim2 += 1
lim1 = lim2
while E[lim1] > E1:
    lim1 += 1
pl = pl - pl[lim2:lim1].sum()/(lim1-lim2)
pl_norm = pl/pl.max()

plt.close()
fig, ax = plt.subplots()
ax.plot(E, pl_norm)

# Ticks" parameters
xmin = 2
xmax = 2.7 #for xlim
xstep_maj = .1 #for xticks
xstep_min = .05
majorLocator = mp.ticker.MultipleLocator(xstep_maj)
minorLocator = mp.ticker.MultipleLocator(xstep_min)
#ymin = 0
#ymax = 1.15

# Fonts" size
titlesize = 24
labelsize = 18
ticksize = 14
legendsize = 20

# Axes" limits
ax.set_xlim(xmin, xmax)
#ax.set_ylim(ymin, ymax)

# Titles
ax.set_title(r"PL(E)", fontsize=titlesize, verticalalignment="bottom")
ax.set_xlabel("Energy (eV)", fontsize=labelsize)
ax.set_ylabel("PL (normalized)", fontsize=labelsize, verticalalignment="bottom")

# Ticks
xticks = np.arange(xmin, xmax, xstep_min)
ax.set_xticks(xticks, minor=True)
ax.tick_params(right=False, which="both", labelsize=ticksize)
ax.tick_params(which="major", width=.90, length=5)
ax.tick_params(which="minor", width=.75, length=3.5)
ax.xaxis.set_major_locator(majorLocator)
ax.xaxis.set_minor_locator(minorLocator)

# Legend
labels=["data"]
ax.legend(labels, fontsize=legendsize)

fig.tight_layout(rect=(0, 0, 0.95, 0.95))

plt.show()

## Plot of the PL as a function of wavelength and energy

In [8]:
def wavelen2E(wavelen):
    return ((h*c) / (wavelen*1e-9)) / e # with wavelen in nm

def E2wavelen(E):
    w = E.copy()
    near_zero = np.isclose(w, 0)
    w[near_zero] = np.inf
    w[~near_zero] = ((h*c) / (E[~near_zero]*e)) * 1e9 # in nm
    return w

plt.close()
fig, ax_w = plt.subplots()
ax_w.plot(wavelen, pl_norm)

ax_E = ax_w.secondary_xaxis('top', functions=(wavelen2E, E2wavelen))
#ax_E.set_xlabel('Energy (eV)')

plt.tight_layout(rect=(0, 0, 0.98, 0.98))
plt.show()

# Plot all files in a directory

## Plot the spectra as a function of wavelength

In [65]:
plt.close("all")
fig, ax = plt.subplots()
axs = []

# The signal will be averaged between lambda1 and lambda2 to calculate the background
lambda1 = 880
lambda2 = 960
bg_correction = True #  for correcting the background or not

# Data directory's path
dir_path = r"pl files"
# Saving directory's path
save_loc = r"figures"
#save_figs = input("save the plots? y/n (default is no)")
save_figs = "no"

i = 1
for filename in os.listdir(dir_path):
    if filename.split(".")[0] == filename: # if it's a directory and not a file
        continue
    if filename[0] == ".": # if it's a cache file
        continue
        
    filepath = dir_path + r"/" + filename
    data = pd.read_csv(filepath, sep=r"\t", header=None, index_col=False, names=["Wavelength", "Intensity"],
                      engine="python")
    wavelen = data["Wavelength"]
    pl = data["Intensity"]
    
    if bg_correction == True:
        lim1 = 1
        while wavelen[lim1] < lambda1:
            lim1 += 1
        lim2 = lim1
        while wavelen[lim2] < lambda2:
            lim2 += 1
        pl = pl - pl[lim1:lim2].sum()/(lim2-lim1)
    pl_norm = pl/pl.max()
    
    #fig, ax = plt.subplots(figsize=(8,6))
    #axs.append(ax)
    ax.plot(wavelen, pl_norm, label=filename.split(".")[0])
    fig.tight_layout(rect=(0, 0, 0.95, 0.95))
    ax.legend()
    
    if save_figs.lower() == "yes":
        os.chdir(save_loc)
        fig.savefig(filename.split(".")[0] + ".png")
        plt.close()
        os.chdir(path_dir)
    
    k2 = 0
    while pl[k2] != pl.max():
        k2 = k2+1
        
    print(i, " : ", filename, ", ", "peak position : ", round(wavelen[k2], 1), " nm", sep="")
    
    i = i + 1

plt.show()

1 : PL_data_1.txt, peak position : 784.3 nm
2 : PL_data_2.txt, peak position : 779.8 nm
3 : PL_data_3.txt, peak position : 641.7 nm
4 : PL_data_6.txt, peak position : 759.3 nm
5 : PL_data_4.txt, peak position : 713.8 nm
6 : PL_data_5.txt, peak position : 726.2 nm


## Plot the spectra as a function of Energy

In [64]:
plt.close("all")
fig, ax = plt.subplots()
axs = []

h = 6.626e-34
c = 3e8
eV = 1.6e-19

# The signal will be averaged between lambda1 and lambda2 to calculate the background
E1 = 1.30
E2 = 1.35
bg_correction = True #  for correcting the background or not

# Data directory's path
dir_path = r"pl files"
# Saving directory's path
save_loc = r"figures"
#save_figs = input("save the plots? y/n (default is no)")
save_figs = "no"

i = 1
for filename in os.listdir(dir_path):
    if filename.split(".")[0] == filename: # if it's a directory and not a file
        continue
    if filename[0] == ".": # if it's a cache file
        continue
        
    filepath = dir_path + r"/" + filename
    wavelen, pl = np.loadtxt(filepath).T
    E = (h*c/(wavelen*1e-9))/eV
    E = E[::-1]
    pl = pl[::-1]

    if bg_correction == True:
        lim1 = 1
        while E[lim1] < E1:
            lim1 += 1
        lim2 = lim1
        while E[lim2] < E2:
            lim2 += 1
        pl = pl - pl[lim1:lim2].sum()/(lim2-lim1)
    pl_norm = pl/pl.max()
    
    #fig, ax = plt.subplots(figsize=(8,6))
    #axs.append(ax)
    ax.plot(E, pl_norm, label=filename.split(".")[0])
    fig.tight_layout(rect=(0, 0, 0.95, 0.95))
    ax.legend()
    
    if save_figs.lower() == "yes":
        os.chdir(save_loc)
        fig.savefig(filename.split(".")[0] + ".png")
        plt.close()
        os.chdir(path_dir)
    
    k2 = 0
    while pl[k2] != pl.max():
        k2 = k2+1
        
    print(i, " : ", filename, ", ", "peak position : ", round(E[k2], 2), " eV", sep="")
    
    i = i + 1

plt.show()

1 : PL_data_1.txt, peak position : 1.58 eV
2 : PL_data_2.txt, peak position : 1.59 eV
3 : PL_data_3.txt, peak position : 1.94 eV
4 : PL_data_6.txt, peak position : 1.64 eV
5 : PL_data_4.txt, peak position : 1.74 eV
6 : PL_data_5.txt, peak position : 1.71 eV


In [54]:
E[lim1+1]

1.2992034584772536

# Use of a cursor

In [120]:
cursor = Cursor(plt.gca(), color="red", lw=1)

plt.show()